In [1]:
import cv2 as cv
import datetime
import numpy as np
import os

In [2]:
def video_frames(video_file):
    assert os.path.isfile(video_file)
    video = cv.VideoCapture(video_file)
    yesh = True
    while yesh:
        yesh, frame = video.read()
        if yesh:
            yield frame
    video.release()

In [3]:
PUNCH_OUT_COLOR = 1

In [4]:
def punch_frame(frame, pixel_circs):
    for ((x, y, r), punchout) in pixel_circs:
        mask = np.zeros(frame.shape, np.uint8)
        mask = cv.circle(mask, (x, y), r, (PUNCH_OUT_COLOR, PUNCH_OUT_COLOR, PUNCH_OUT_COLOR), -1)
        if punchout:
            frame = np.where(mask > 0, mask, frame)
        else:
            frame = np.where(mask > 0, frame, mask + PUNCH_OUT_COLOR)
    return frame

In [5]:
FPS  = 120
SIZE = (64, 64)

def punch_video(video_file, style, pixel_circs):
    basename = os.path.basename(video_file)
    now = str(datetime.datetime.now(tz=datetime.timezone.utc))
    punched_stem = os.path.dirname(os.path.dirname(video_file)) + "/" + style + now
    os.makedirs(punched_stem)
    writer = cv.VideoWriter(punched_stem + "/" + basename, cv.VideoWriter_fourcc(*'mp4v'), FPS, SIZE, True)

    for frame in video_frames(video_file):
        writer.write(punch_frame(frame, pixel_circs))

    writer.release()

In [6]:
DISTANCE_TO_SCREEN = 106
SCALE_MOVIE = 15
SCREEN_DIMS = (100, 62)
SCREEN_RES = (1920, 1080)
VIDEO_RES = (64, 64)
SCALED_VIDEO_RES = (VIDEO_RES[0] * SCALE_MOVIE, VIDEO_RES[1] * SCALE_MOVIE)
VIDEO_THRESHOLDS = [
    [(SCREEN_RES[0] - SCALED_VIDEO_RES[0]) / 2, (SCREEN_RES[1] - SCALED_VIDEO_RES[1]) / 2],
    [(SCREEN_RES[0] + SCALED_VIDEO_RES[0]) / 2, (SCREEN_RES[1] + SCALED_VIDEO_RES[1]) / 2]
]

In [7]:
SCREEN_HALFWIDTH_DEGREES = np.degrees(np.arctan(SCREEN_DIMS[0] / 2 / DISTANCE_TO_SCREEN))
SCREEN_HALFHEIGHT_DEGREES = np.degrees(np.arctan(SCREEN_DIMS[1] / 2 / DISTANCE_TO_SCREEN))

In [8]:
def degrees_to_pixels(theta):
    return theta * SCREEN_RES[0] / (2 * SCREEN_HALFWIDTH_DEGREES)

In [9]:
# LOWER PIXEL X => LEFTER
# LOWER PIXEL Y => HIGHER
def circle_to_pixels(x, y, r):
    x = SCREEN_RES[0] / 2 + degrees_to_pixels(x)
    y = SCREEN_RES[1] / 2 - degrees_to_pixels(y)
    return x, y, degrees_to_pixels(r)

In [10]:
HOLD_DEGREES = 3

In [11]:
RFS = [((5, 5, 2), True)]

In [12]:
def rfs_to_pixels(rfs):
    for ((x, y, r), p) in rfs:
        x, y, r = circle_to_pixels(x, y, r + HOLD_DEGREES)
        x = (x - VIDEO_THRESHOLDS[0][0]) / SCALE_MOVIE
        y = (y - VIDEO_THRESHOLDS[0][1]) / SCALE_MOVIE
        r /= SCALE_MOVIE
        if x >= 0 and x < VIDEO_RES[0] and y >= 0 and y < VIDEO_RES[1]:
            yield ((round(x), round(y), round(r)), p)

In [13]:
punch_video("/home/eli/Documents/MATLAB/Passive Moving MNIST/full/moving_mnist0.mp4", "punch_trial",  list(rfs_to_pixels(RFS)))